In [27]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import pandas.io.data as web
import datetime
import tensorflow as tf
from sklearn import svm, preprocessing
from keras.layers.normalization import BatchNormalization
import talib
from sklearn.preprocessing import MinMaxScaler


In [2]:
class KOSPIDATA:
    def __init__(self):
        start = datetime.datetime(1998, 5, 1)
        end = datetime.datetime(2016, 12, 31)
        kospi = web.DataReader("^KS11", "yahoo", start, end)

        self.arr_date = np.array(kospi.index)
        self.arr_open = np.array(kospi.Open, dtype=float)
        self.arr_close = np.array(kospi['Adj Close'], dtype=float)
        self.arr_high = np.array(kospi.High, dtype=float)
        self.arr_low = np.array(kospi.Low, dtype=float)
        self.arr_volume = np.array(kospi.Volume, dtype=float)


    def MA5(self):
        ma5 = talib.MA(self.arr_close, timeperiod=5)
        return ma5

    def MA10(self):
        ma10 = talib.MA(self.arr_close, timeperiod=10)
        return ma10

    def StoK(self):
        fastk, fastd= talib.STOCHF(self.arr_high, self.arr_low,self.arr_close, fastk_period=15, fastd_matype=0)
        slowk, slowd = talib.STOCH(self.arr_high, self.arr_low, self.arr_close, fastk_period= 15, slowk_period=5, slowk_matype=0, slowd_period=3, slowd_matype=0)
        return fastk, fastd, slowd

    def Momentum(self):
        momentum = self.arr_close - np.roll(self.arr_close,4)
        return momentum

    def ROC(self):
        roc = (self.arr_close / np.roll(self.arr_close,4)) * 100
        return roc


    def AD_Oscil(self):
        ad_oscil = (self.arr_high - np.roll(self.arr_close,1))/(self.arr_high - self.arr_low)
        return ad_oscil

    def DIsp5(self):
        disp5 = (self.arr_close/talib.MA(self.arr_close, timeperiod =5))*100
        return disp5

    def DIsp10(self):
        disp10 = (self.arr_close/talib.MA(self.arr_close, timeperiod =10))*100
        return disp10

    def OSCP(self):
        oscp = (self.MA5()-self.MA10())/self.MA5()
        return oscp

    def CCI(self):
        cci = talib.CCI(self.arr_high, self.arr_low, self.arr_close, timeperiod=7)
        return cci

    def RSI(self):
        rsi = talib.RSI(self.arr_close, timeperiod =4)
        return rsi

In [4]:
K = KOSPIDATA()

    # FEATURES = ['high', 'low', 'open', 'close', 'volume']
FEATURES = ['high', 'low', 'open', 'close','ma5', 'ma10', 'fastk', 'fastd', 'slowd','momentum','roc','ad_oscil','disp5','disp10', 'oscp','cci','rsi']

data = {'year': K.arr_date,
            'open': K.arr_open,
            'high': K.arr_high,
            'low': K.arr_low,
            'close': K.arr_close}

df = DataFrame(data, columns=['year', 'high', 'low', 'open', 'close','ma5', 'ma10', 'fastk', 'fastd', 'slowd','momentum','roc','ad_oscil','disp5','disp10', 'oscp','cci','rsi'])


In [33]:
df.fastk, df.fastd, df.slowd = K.StoK()
df.ad_oscil = K.AD_Oscil()
df.ad_oscil[0] = np.NaN
df.momentum = K.Momentum()
df.momentum[0:4] = np.NaN
df.roc = K.ROC()
df.ma5, df.ma10 ,df.disp5, df.disp10 = K.MA5(), K.MA10(), K.DIsp5(), K.DIsp10()
df.oscp , df.cci = K.OSCP(), K.CCI()
df.rsi = K.RSI()

C:\Users\UCIM-WORKSTATION\Anaconda64\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\UCIM-WORKSTATION\Anaconda64\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [50]:
df.ix[:20].isnull().sum()

year         0
high         0
low          0
open         0
close        0
ma5          4
ma10         9
fastk       16
fastd       16
slowd       20
momentum     4
roc          0
ad_oscil     1
disp5        4
disp10       9
oscp         9
cci          6
rsi          4
dtype: int64

In [11]:
def prediction():
    profit = []
    for i in range(len(df['close']) - 1):
        if df['close'][i] < df['close'][i + 1]:
            profit.append('1')
        else:
            profit.append('0')
    return profit

In [20]:
profit = np.array(prediction())
profit = np.append(profit, np.NaN)

In [22]:
profit.shape

(4618,)

In [73]:
full_X = np.array(df[FEATURES].values[20:-2, :])
print(full_X.shape)
print(full_X)
full_Y = profit[20:-2]
print(full_Y, full_Y.shape)

(4596, 17)
[[  3.35480011e+02   3.23670013e+02   3.35480011e+02 ...,  -2.90789370e-02
    1.16482949e+01   3.60321540e+01]
 [  3.32609985e+02   3.24820007e+02   3.24820007e+02 ...,  -6.15856994e-03
    4.20915339e+01   5.07667770e+01]
 [  3.41529999e+02   3.33369995e+02   3.34010010e+02 ...,   1.11551880e-02
    9.53466804e+01   6.30716708e+01]
 ..., 
 [  2.03770996e+03   2.03154004e+03   2.03617004e+03 ...,   5.36802158e-04
   -8.09333152e+01   5.29579424e+01]
 [  2.03976001e+03   2.03480005e+03   2.03832996e+03 ...,  -4.17153129e-05
   -5.70352280e+01   6.06904756e+01]
 [  2.04443005e+03   2.03712000e+03   2.04073999e+03 ...,  -3.22878921e-04
    3.76591314e+01   7.42002933e+01]]
['1' '1' '1' ..., '1' '1' '0'] (4596,)


In [77]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=17, init='uniform', activation='relu'))
# model.add(BatchNormalization())
model.add(Dense(8, init='uniform', activation='relu'))
# model.add(BatchNormalization())
model.add(Dense(6, init='uniform', activation='relu'))
# model.add(BatchNormalization())
model.add(Dense(1, init='uniform', activation='sigmoid'))
# model.add(BatchNormalization())

C:\Users\UCIM-WORKSTATION\Anaconda64\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, activation="relu", input_dim=17, kernel_initializer="uniform")`
  app.launch_new_instance()
C:\Users\UCIM-WORKSTATION\Anaconda64\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
C:\Users\UCIM-WORKSTATION\Anaconda64\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="uniform")`
C:\Users\UCIM-WORKSTATION\Anaconda64\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`


In [78]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(full_X, full_Y, validation_split=0.25, nb_epoch=150, batch_size=10)
# evaluate the model
scores = model.evaluate(full_X, full_Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

C:\Users\UCIM-WORKSTATION\Anaconda64\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3447 samples, validate on 1149 samples
Epoch 1/150
3447/3447 [==============================] - 2s - loss: 0.6920 - acc: 0.5393 - val_loss: 0.7201 - val_acc: 0.5065
Epoch 2/150
3447/3447 [==============================] - 1s - loss: 0.6908 - acc: 0.5393 - val_loss: 0.7201 - val_acc: 0.5065
Epoch 3/150
3447/3447 [==============================] - 1s - loss: 0.6907 - acc: 0.5393 - val_loss: 0.7210 - val_acc: 0.5065
Epoch 4/150
3447/3447 [==============================] - 1s - loss: 0.6905 - acc: 0.5393 - val_loss: 0.7201 - val_acc: 0.5065
Epoch 5/150
3447/3447 [==============================] - 1s - loss: 0.6903 - acc: 0.5393 - val_loss: 0.7210 - val_acc: 0.5065
Epoch 6/150
3447/3447 [==============================] - 1s - loss: 0.6903 - acc: 0.5393 - val_loss: 0.7207 - val_acc: 0.5065
Epoch 7/150
3447/3447 [==============================] - 1s - loss: 0.6903 - acc: 0.5393 - val_loss: 0.7207 - val_acc: 0.5065
Epoch 8/150
3447/3447 [==============================] - 1s - loss: 0.

KeyboardInterrupt: 